# ISI-DDBB-RISK

Este cuadernillo implementará la estructura del juego del RISK, en base de datos. Además esté cuadernillo pretende hacer visible el desarrollo paso a paso de la implementación, no solo necesaria para el juego, sino también para mantener un sistema de consultas de partidas de RISK.Todos los diagramas E/R que vayan apareciendo son sucteptibles de sufrir cambios a lo largo del proceso

In [1]:
%load_ext sql

ImportError: No module named 'sql'

In [3]:
%sql sqlite:///ISI-RISK-v2.db

'Connected: None@ISI-RISK-v2.db'

## Tabla de Jugadores

![Diagrama E/R](IMG/tabla_jugadores_v2.png)

In [4]:
%%sql
CREATE TABLE jugadores(nombre VARCHAR(20),
                       tipo VARCHAR(20),
                       PRIMARY KEY(nombre));

(sqlite3.OperationalError) table jugadores already exists [SQL: 'CREATE TABLE jugadores(nombre VARCHAR(20),\n                       tipo VARCHAR(20),\n                       PRIMARY KEY(nombre));']


In [11]:
%%sql
INSERT INTO jugadores(nombre,tipo) VALUES('Jorge','humano');
INSERT INTO jugadores(nombre,tipo) VALUES('Ana','IA');
INSERT INTO jugadores(nombre,tipo) VALUES('Peter','IA');
INSERT INTO jugadores(nombre,tipo) VALUES('Maria','humano');

1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

## Tabla de Colores

![Diagrama E/R](IMG/tabla_colores_v2.png)

In [6]:
%%sql
CREATE TABLE colores(color VARCHAR(20),
                     PRIMARY KEY(color));

Done.


[]

In [12]:
%%sql
INSERT INTO colores(color) VALUES('azul');
INSERT INTO colores(color) VALUES('verde');
INSERT INTO colores(color) VALUES('rojo');
INSERT INTO colores(color) VALUES('naranja');

1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

## Tabla de Partidas

![Diagrama E/R](IMG/tabla_partidas_v2.png)

In [7]:
%%sql
CREATE TABLE partidas(inicio DATETIME,
                      fin DATETIME,
                      PRIMARY KEY(inicio,fin));

Done.


[]

In [13]:
%%sql
INSERT INTO partidas(inicio,fin) VALUES('03/12/2016','03/12/2016');
INSERT INTO partidas(inicio,fin) VALUES('13/12/2016','13/12/2016');
INSERT INTO partidas(inicio,fin) VALUES('10/12/2016','10/12/2016');
INSERT INTO partidas(inicio,fin) VALUES('04/12/2016','04/12/2016');

1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

## Relación Jugadores,Colores y Partidas 

Un jugador es un color para una partida. Son los jugadores por partidas.

![Diagrama E/R](IMG/relacion_jugadores_colores_partidas.png)

In [11]:
%%sql
DROP TABLE jugadores;
CREATE TABLE jugadores(nombre VARCHAR(20),
                       tipo VARCHAR(20),
                       color VARCHAR(20) NOT NULL,
                       PRIMARY KEY(nombre,color));

Done.
Done.


[]

In [12]:
%%sql
INSERT INTO jugadores(nombre,tipo,color) VALUES('Jorge','humano','verde');
INSERT INTO jugadores(nombre,tipo,color) VALUES('Ana','IA','rojo');
INSERT INTO jugadores(nombre,tipo,color) VALUES('Peter','IA','azul');
INSERT INTO jugadores(nombre,tipo,color) VALUES('Maria','humano','blanco');

1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [13]:
%%sql
DROP TABLE partidas;
CREATE TABLE partidas(inicio DATETIME,
                      fin DATETIME,
                      nombre VARCHAR(20),
                      PRIMARY KEY(inicio,fin,nombre),
                      FOREIGN KEY (nombre) REFERENCES jugadores (nombre));

Done.
Done.


[]

In [14]:
%%sql
INSERT INTO partidas(inicio,fin,nombre) VALUES('03/12/2016','03/12/2016','Jorge');
INSERT INTO partidas(inicio,fin,nombre) VALUES('03/12/2016','03/12/2016','Maria');
INSERT INTO partidas(inicio,fin,nombre) VALUES('10/12/2016','10/12/2016','Jorge');
INSERT INTO partidas(inicio,fin,nombre) VALUES('10/12/2016','10/12/2016','Ana');

1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

Permitiendonos así eliminar fisicamente la tabla de colores, pero seguir manteniendo la relación de que jugador uso que color en que partida.

## Tabla Jugadas

![Diagrama E/R](IMG/tabla_turnos_v2.png)

In [15]:
%%sql
DROP TABLE jugadas;
CREATE TABLE jugadas(id INT,
                    batallones INT,
                    movimiento VARCHAR(20),
                    PRIMARY KEY(id));

Done.
Done.


[]

In [16]:
%%sql
INSERT INTO jugadas(id,batallones,movimiento) VALUES(1,0,"ataco");
INSERT INTO jugadas(id,batallones,movimiento) VALUES(2,-2,"defiendo");
INSERT INTO jugadas(id,batallones,movimiento) VALUES(3,-1,"ataco");
INSERT INTO jugadas(id,batallones,movimiento) VALUES(4,0,"ataco");

1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

## Relación Jugadores, Jugadas y Partidas

Turnos relizados en una partida por un jugador.

![Diagrama E/R](IMG/relacion_jugadores_partidas_jugadas.png)

Esta relación la podríamos representar de dos formas: 
    1.- Creando una tabla aparte jugadas_realizadas que contenga los IDs de partidas y el ID de jugadas
    2.- A la tabla de jugadas añadirle los campos de partidas,y que sean estos a su vez IDs. Esto nos permitiría llevar un orden sobre como se realizaron las jugadas en una partida de manera más especifica, aunque en la anterior también está esta relación de en que orden en función del ID más alto. Esta no la vamos a elegir ya que mepezaríamos a hacer unos IDs demasiado largos.


In [29]:
%%sql
DROP TABLE jugadas_realizadas;
CREATE TABLE jugadas_realizadas(inicio DATETIME,
                                fin DATETIME,
                                nombre VARCHAR(20),
                                id INT,
                                PRIMARY KEY (id),
                                FOREIGN KEY (id) REFERENCES jugadas(id) ON DELETE CASCADE);

Done.
Done.


[]

In [30]:
%%sql
INSERT INTO jugadas_realizadas(inicio,fin,nombre,id) VALUES('03/12/2016','03/12/2016','Jorge',1);
INSERT INTO jugadas_realizadas(inicio,fin,nombre,id) VALUES('10/12/2016','10/12/2016','Ana',2);
INSERT INTO jugadas_realizadas(inicio,fin,nombre,id) VALUES('03/12/2016','03/12/2016','Maria',3);
INSERT INTO jugadas_realizadas(inicio,fin,nombre,id) VALUES('10/12/2016','10/12/2016','Jorge',4);

1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

## Tabla de Paises

![Diagrama E/R](IMG/tabla_paises_v2.png)

In [15]:
%%sql
DROP TABLE paises;
CREATE TABLE paises(nombre VARCHAR(20),
                    vecinos VARCHAR(100),
                    PRIMARY KEY(nombre));

Done.
Done.


[]

In [16]:
%%sql
INSERT INTO paises(nombre,vecinos) VALUES('España','Francia,Portugal,Inglaterra');
INSERT INTO paises(nombre,vecinos) VALUES('Portugal','España');
INSERT INTO paises(nombre,vecinos) VALUES('Inglaterra','España');
INSERT INTO paises(nombre,vecinos) VALUES('Francia','Alemania,Italia,España');

1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

## Agregación de Paises a los Turnos

Colonización de los Paises según el turno 

![Diagrama E/R](IMG/relacion_entre_tablas.png)

Podríamos representar la agregación como un campo que sea país colonizado en la tablas de jugadas realizadas, también podríamos manejar una tabla de colonizaciones que realiciones la jugada con los paises, por simplificar el manejo de las tablas haremos una tabla más que solo maneje en que jugada se colonizo que país.

In [31]:
%%sql
CREATE TABLE colonizaciones(id INT,
                            pais VARCHAR(20) NOT NULL,
                            PRIMARY KEY (id));

Done.


[]

In [32]:
%%sql
INSERT INTO colonizaciones(id,pais) VALUES(1,'España');
INSERT INTO colonizaciones(id,pais) VALUES(2,'Italia');
INSERT INTO colonizaciones(id,pais) VALUES(3,'Francia')

1 rows affected.
1 rows affected.
1 rows affected.


[]

## Querys

### 1.- Batallones del jugador en la partida X en el momento Y de la partida

In [ ]:
%%sql SELECT COUNT(batallones) FROM jugadas J
JOIN jugadas_realizadas JR  ON J.id = JR.id
WHERE JR.inicio = "" AND JR.fin = ""

### 2.- Jugadores en una partida

In [ ]:
%%sql SELECT * FROM partidas WHERE inicio = "" AND fin = ""

### 3.- Total de paises conquistados por un jugador

In [ ]:
%%sql SELECT * FROM jugadas_realizadas JR WHERE nombre = "" 
JOIN colonizaciones C ON JR.id = C.id

### 4.-Paises conquistados por un jugador en una partida

In [ ]:
%sql SELECT * FROM jugadas_realizadas JR
WHERE (nombre = "" AND inicio = "" AND fin = "") 
JOIN colonizaciones C ON JR.id = C.id

### 5.- Batallones con los que acabo un jugador en una partida

In [3]:
%%sql SELECT batallones FROM jugadas J
JOIN jugadas_realizadas JR  ON J.id = JR.id
WHERE (JR.inicio = "" AND JR.fin = "")
AND JR.nombre = "" 

ERROR:root:Cell magic `%%sql` not found.


### 6.- Total colores con los ha jugado un jugador

In [ ]:
%%sql SELECT COUNT(*) FROM jugadores WHERE jugador = "Jorge"

### 7.- Total de partidas jugadas por un jugador

In [ ]:
%%sql SELECT COUNT(*) FROM partidas WHERE jugador="Jorge";

### 8.- Total de jugadores que han usado un color en un rango de partidas

In [ ]:
%% sql SELECT * FROM jugadores J WHERE color = ""
JOIN partidas P ON P.nombre = J.nombre 
AND (P.inicio = "" AND P.fin = "")
OR (P.inicio = "" AND P.fin = "")